In [7]:
#!pip install geopandas
## if above does not work, try following.
#pip install git+git://github.com/geopandas/geopandas.git

import geopandas as gpd
import pandas as pd
import re
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
import numpy as np


In [16]:
GeoFile = 'crimes-in-boston/Boston_Neighborhoods.shx'
boston_geo = gpd.read_file(GeoFile)

In [13]:
type(boston_geo)

geopandas.geodataframe.GeoDataFrame

In [17]:
df1 = pd.DataFrame(boston_geo)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 1 columns):
geometry    26 non-null geometry
dtypes: geometry(1)
memory usage: 336.0 bytes


In [18]:
location=[]
pattern = re.compile(r'\d{2}\.\d+')
for i in df1['geometry']:
    matches = pattern.findall(str(i))
    for i in matches:            
        location.append(i)
len(location)  

64222

In [6]:
#location

In [19]:
lat=[]
long=[]
count = 0
for i in location:
    if count % 2 == 1:
        lat.append(float(i))
    else:
        long.append(0-float(i))
    count += 1

In [128]:
location_df = {'lat':lat,'long':long}
df = pd.DataFrame(location_df) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32111 entries, 0 to 32110
Data columns (total 2 columns):
lat     32111 non-null float64
long    32111 non-null float64
dtypes: float64(2)
memory usage: 501.9 KB


In [129]:
df.head()

,lat,long
0,42.272013,-71.125927
1,42.272347,-71.125748
2,42.272483,-71.125664
3,42.272579,-71.125551
4,42.272491,-71.125729


In [130]:
#The distict Boston shapefile including all bostion lat&long
df.drop_duplicates(keep=False,inplace=True) 
df.info()
#df.to_csv('C:/Users/shuji/Desktop/Python project/BostonCrimeProject/crimes-in-boston/bostonshapefile.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25013 entries, 1883 to 32109
Data columns (total 2 columns):
lat     25013 non-null float64
long    25013 non-null float64
dtypes: float64(2)
memory usage: 586.2 KB


In [23]:
df1 = df[(df.long <= -71.15)]
boston = folium.Map(location = [df1['lat'].mean(), 
                                  df1['long'].mean()], 
                      zoom_start = 12)
mc = MarkerCluster()
#creating a Marker for each point. 
for row in df1.itertuples():
    mc.add_child(folium.Marker(location = [row.lat, row.long]))

boston.add_child(mc)

boston

In [24]:
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    miles = 6367 * c/1.609
    return miles

In [25]:
crime = 'crimes-in-boston/crimes_over_7_unique.csv'
crime = pd.read_csv(crime)
df2 = pd.DataFrame(crime) 
longc_list = [i for i in df2['Long']]
latc_list = [i for i in df2['Lat']]
long_list = [i for i in df['long']]
lat_list = [i for i in df['lat']]

In [26]:
remove_long = []
remove_lat = []
for i in range(len(longc_list)):
    for j in range(len(long_list)):
        if haversine_np(longc_list[i],latc_list[i],long_list[j],lat_list[j])<0.5:
            remove_long.append(long_list[j])
            remove_lat.append(lat_list[j])

In [27]:
len(remove_long)

7092

In [131]:
##df4 is the distinct lat&long that should be removed from Boston shapefile.
unique_lat = [] 
unique_long = []
[unique_lat.append(x) for x in remove_lat if x not in unique_lat]
[unique_long.append(x) for x in remove_long if x not in unique_long]
df3 = {'lat':unique_lat,'long':unique_long}
df4 = pd.DataFrame(df3) 
df4.info()
#df4.to_csv('C:/Users/shuji/Desktop/Python project/BostonCrimeProject/crimes-in-boston/crime_area_lat&long_from_boston_shapefile.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5431 entries, 0 to 5430
Data columns (total 2 columns):
lat     5431 non-null float64
long    5431 non-null float64
dtypes: float64(2)
memory usage: 85.0 KB


In [132]:
#df4.head()
df4.shape

(5431, 2)

In [133]:
#df.head()
df.shape

(25013, 2)

In [134]:
result = pd.concat([df, df4])
result.shape

(30444, 2)

In [135]:
sum(result.duplicated())

5431